In [3]:
import os
os.chdir(r'C:\Users\DianaEspinosaLozano\anaconda3\envs\thesis\Lib\site-packages\bslib')
import bslib as bsl
import pandas as pd


In [4]:
# Example for generic DC Battery
BAT = bsl.DCBatMod(system_id="S5")
b = bsl.load_parameters(system_id ="S5")
b

AttributeError: module 'bslib' has no attribute 'DCBatMod'

In [ ]:
# Simulate One Time Step
p_load = 750        # W (positive = charging, negative = discharging)
results = BAT.simulation(p_load=p_load,
                         p_pv = 500, # PV vorhanden für BS
                         soc=0.5,
                         dt=60)
results

In [ ]:
# Simulate Time Series
# pv - Eigenverbrauch -> BS, wenn BS voll dann Einspeisung
# read elecdemandhourly240723.csv
df = pd.read_csv('C:\\Users\\DianaEspinosaLozano\\anaconda3\\envs\\thesis\\elecdemandhourly240723.csv')
# p_pv = df['pv']
# p_load = df['Eigenverbrauch']

# Assuming df is your DataFrame with columns 'p_load' and 'p_pv'
initial_soc = 0.5  # Initial state of charge
dt = 3600  # Time step, for example, 1 hour

soc = initial_soc
results = []

for _, row in df.iterrows():
    result = BAT.simulation(p_load=row['h0_dyn'], p_pv=row['pv'], soc=soc, dt=dt)
    soc = result.soc  # Update soc for the next iteration
    results.append({'p_pvbs': result.p_pvbs, 'p_bat': result.p_bat, 'soc': soc})

# Convert results to a DataFrame
df_results = pd.DataFrame(results)
df = pd.concat([df, df_results], axis=1)
df

In [ ]:
import matplotlib.pyplot as plt
df_plot = df[['pv', 'Eigenverbrauch', 'p_pvbs']]
ax = df_plot.plot()
ax.set_xlabel("Date")
ax.set_ylabel("Power [kWh]")
plt.show()

In [ ]:
# 1 Tag: 1.1.2016
tag = df.loc[0:24, ['pv', 'Eigenverbrauch', 'soc']]
tag_plot = tag.plot()
tag_plot.set_xlabel('Time (h)')
tag_plot.set_ylabel('Power (kWh)')
plt.show

In [9]:
# AC Battery

# Example for generic AC Battery
p_inv = 1000    # W
e_bat = 1       # kWh
BATAC = bsl.ACBatMod(system_id="SG1",
                   p_inv_custom=p_inv,
                   e_bat_custom=e_bat)

p_load = 750        # W (positive = charging, negative = discharging)
results = BATAC.simulate(p_load=p_load,
             soc=0.5,
             dt=60)
results

AttributeError: module 'bslib' has no attribute 'ACBatMod'

In [ ]:
df_ac = pd.read_csv('C:\\Users\\DianaEspinosaLozano\\anaconda3\\envs\\thesis\\elecdemandhourly240723.csv')

initial_soc = 0.5  # Initial state of charge
dt = 3600  # Time step, for example, 1 hour

soc = initial_soc
results = []

for _, row in df_ac.iterrows():
    result = BATAC.simulate(p_load=row['Eigenverbrauch'], p_pv=row['pv'], soc=soc, dt=dt)
    soc = result.soc  # Update soc for the next iteration
    results.append({'p_pvbs': result.p_pvbs, 'p_bat': result.p_bat, 'soc': soc})

# Convert results to a DataFrame
df_results = pd.DataFrame(results)
df = pd.concat([df, df_results], axis=1)
df